# c06数据加载，存储与文件格式

In [111]:
import numpy as np
import pandas as pd
import sys
import csv
import json
from lxml import objectify
from io import StringIO
import requests
import sqlite3
import sqlalchemy as sqla

In [2]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [3]:
df = pd.read_csv('examples/ex1.csv')

df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
pd.read_csv('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [6]:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
names = ['a', 'b', 'c', 'd', 'message']

pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [9]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [10]:
parsed = pd.read_csv('examples/csv_mindex.csv',
                       index_col=['key1', 'key2'])

parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [11]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [12]:
result = pd.read_csv('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [13]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [14]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [15]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [16]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [17]:
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [18]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [19]:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [20]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [21]:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


In [22]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [23]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

In [24]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [25]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

In [26]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [27]:
data.to_csv('examples/out.csv')

In [28]:
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [29]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [30]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [31]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [32]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [33]:
dates = pd.date_range('1/1/2000', periods=7)

ts = pd.Series(np.arange(7), index=dates)

ts.to_csv('examples/tseries.csv', header=False)

!cat examples/tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


In [34]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [35]:
f = open('examples/ex7.csv')

reader = csv.reader(f)


In [36]:
for line in reader:
    print(line)
    

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [37]:
with open('examples/ex7.csv') as f1:
    lines = list(csv.reader(f1))
    
header, values = lines[0], lines[1:]

data_dict = {h: v for h, v in zip(header,zip(*values))}

data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [38]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL


In [39]:
#f = open('examples/ex7.csv')

reader = csv.reader(f, dialect=my_dialect)

In [40]:
reader = csv.reader(f, delimiter='|')

In [41]:
with open('mydata.csv', 'w') as f2:
    writer = csv.writer(f2, dialect=my_dialect)
    writer.writerow(('onw', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [42]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [43]:
result = json.loads(obj)

result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [44]:
asjson = json.dumps(result)

asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [45]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])

siblings

,name,age
0,Scott,30
1,Katie,38


In [46]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [47]:
data = pd.read_json('examples/example.json')

data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [48]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [49]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


In [50]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

len(tables)

1

In [51]:
failures = tables[0]

failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [52]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

In [53]:
path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [54]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [55]:
perf = pd.DataFrame(data)

perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


In [56]:
tag = '<a href="http://www.google.com">Google</a>'

root = objectify.parse(StringIO(tag)).getroot()

root

<Element a at 0x110283148>

In [57]:
root.get('href')

'http://www.google.com'

In [58]:
root.text

'Google'

In [59]:
#6.2

In [60]:
frame = pd.read_csv('examples/ex1.csv')

frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [61]:
frame.to_pickle('examples/frame_pickle')

In [62]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [70]:

frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [71]:
store['obj1']

,a
0,-0.988452
1,-1.242485
2,0.874260
3,-1.183504
4,-0.837447
5,0.835853
6,-0.821926
7,1.764246
8,0.601695
9,0.020497


In [72]:
store.put('obj2', frame, format='table')

store.select('obj2', where=['index >= 10 and index <= 15'])



,a
10,0.159202
11,1.705783
12,0.610281
13,0.627853
14,-0.621970
15,0.655521


In [73]:
store.close()

In [75]:
frame.to_hdf('mydata.h5', 'obj3', format='table')

pd.read_hdf('mydata.h5', 'obj3', where=['index<5'])


,a
0,-0.988452
1,-1.242485
2,0.874260
3,-1.183504
4,-0.837447


In [76]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [77]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')

frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [78]:
writer = pd.ExcelWriter('examples/ex2.xlsx')

frame.to_excel(writer, 'Sheet1')

writer.save()

In [79]:
frame.to_excel('examples/ex2.xlsx')

In [ ]:
#6.3

In [90]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [96]:
data = resp.json()
data

[{'url': 'https://api.github.com/repos/pandas-dev/pandas/issues/26381',
  'repository_url': 'https://api.github.com/repos/pandas-dev/pandas',
  'labels_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/26381/labels{/name}',
  'comments_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/26381/comments',
  'events_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/26381/events',
  'html_url': 'https://github.com/pandas-dev/pandas/pull/26381',
  'id': 443685600,
  'node_id': 'MDExOlB1bGxSZXF1ZXN0Mjc4NTAwOTcy',
  'number': 26381,
  'title': 'BUG: error calculating BusinessHourMixin.apply for long business hour per day',
  'user': {'login': 'jiangyue12392',
   'id': 35633013,
   'node_id': 'MDQ6VXNlcjM1NjMzMDEz',
   'avatar_url': 'https://avatars2.githubusercontent.com/u/35633013?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/jiangyue12392',
   'html_url': 'https://github.com/jiangyue12392',
   'followers_url': 'https://api.github.com/user

In [97]:
data[0]

{'url': 'https://api.github.com/repos/pandas-dev/pandas/issues/26381',
 'repository_url': 'https://api.github.com/repos/pandas-dev/pandas',
 'labels_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/26381/labels{/name}',
 'comments_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/26381/comments',
 'events_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/26381/events',
 'html_url': 'https://github.com/pandas-dev/pandas/pull/26381',
 'id': 443685600,
 'node_id': 'MDExOlB1bGxSZXF1ZXN0Mjc4NTAwOTcy',
 'number': 26381,
 'title': 'BUG: error calculating BusinessHourMixin.apply for long business hour per day',
 'user': {'login': 'jiangyue12392',
  'id': 35633013,
  'node_id': 'MDQ6VXNlcjM1NjMzMDEz',
  'avatar_url': 'https://avatars2.githubusercontent.com/u/35633013?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/jiangyue12392',
  'html_url': 'https://github.com/jiangyue12392',
  'followers_url': 'https://api.github.com/users/jiangyue12392/fo

In [ ]:
data[0]['title']

# 'BUG: error calculating BusinessHourMixin.apply for long business hour per day'

In [92]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,26381,BUG: error calculating BusinessHourMixin.apply...,[],open
1,26380,ENH: Implementing NEP 18's __array_function__,[],open
2,26379,BUG: Series.combine_first with mixed timezones,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
3,26377,Fix type annotations in pandas.core.ops,"[{'id': 1280988427, 'node_id': 'MDU6TGFiZWwxMj...",open
4,26376,pd.read_fwf fails with file pointer to url,[],open
5,26374,BUG: ngroups and len(groups) do not equal when...,"[{'id': 233160, 'node_id': 'MDU6TGFiZWwyMzMxNj...",open
6,26372,Misc Type Annotation fixes,"[{'id': 1280988427, 'node_id': 'MDU6TGFiZWwxMj...",open
7,26370,Feature requests: pandas.DataFrame.quantile wi...,[],open
8,26368,corrwith in 0.24 is much slower than 0.23 (esp...,"[{'id': 8935311, 'node_id': 'MDU6TGFiZWw4OTM1M...",open
9,26367,Pandas import breaks when setting filterwarnings,[],open


In [ ]:
#6.4

In [115]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)

In [116]:
con.commit()

In [117]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [118]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [119]:
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [120]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [121]:
!rm mydata.sqlite